In [1]:
from pyspark.sql import SparkSession

In [2]:
# Spark 세션 생성
spark = SparkSession.builder \
    .appName("PMU Data Query") \
    .config("spark.jars", r"/home/sm/spark/postgresql-42.7.4.jar") \
    .getOrCreate()

your 131072x1 screen size is bogus. expect trouble
24/10/08 17:08:03 WARN Utils: Your hostname, DESKTOP-F6CHKUO resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/10/08 17:08:03 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/08 17:08:04 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
import json


with open("secret.json", "r") as f:
    secret = json.load(f)

In [4]:
# PostgreSQL에서 데이터 읽기

postgres_username = secret.get("POSTGRES_USER", "postgres")
postgres_password = secret.get("POSTGRES_PASSWORD", None)
postgres_port = int(secret.get("POSTGRES_PORT", 5432))
postgres_db = secret.get("POSTGRES_DB", "openpdc")
postgres_server = secret.get("POSTGRES_SERVER", "postgres-server")

jdbc_url = f"jdbc:postgresql://{postgres_server}:{postgres_port}/{postgres_db}"
connection_properties = {
    "user": postgres_username,
    "password": postgres_password,
    "driver": "org.postgresql.Driver"
}

In [6]:
sql = """

select *

from pmudata 

where "timestamp" >= '2024-10-08 17:00:00' and "timestamp" <= '2024-10-08 17:00:01'
limit 10 

"""

In [7]:
postgreTable = spark.read.format("jdbc")\
                .option("url","jdbc:postgresql://{0}:{1}/{2}".format( postgres_server, postgres_port, postgres_db ) )\
                .option("driver", "org.postgresql.Driver")\
                .option("query", sql)\
                .option("user", postgres_username)\
                .option("password", postgres_password)\
                .load()

In [12]:
ps_df = postgreTable.pandas_api()

In [13]:
ps_df

,signalid,key,timestamp,value
0,dba264c2-0ca3-4c42-9e70-3d1b15dff09f,PPA:485,2024-10-04 10:08:10.433,13.069997278737109
1,9b321278-d388-4f4f-aa58-a444540b4d78,PPA:484,2024-10-04 10:08:10.433,0.9995894432067871
2,fa56fefb-3662-42a8-bef1-db44be1606ec,PPA:487,2024-10-04 10:08:10.433,13.039996526581476
3,0d083864-3e6f-4d5d-a01b-823b7ee6ee39,PPA:486,2024-10-04 10:08:10.433,0.9983991980552673
4,c7742ac9-beee-417f-97d0-6476f90b04db,PPA:489,2024-10-04 10:08:10.433,13.01999687891803
5,3d46daf6-e3b9-4515-ab32-948e4358a7e4,PPA:488,2024-10-04 10:08:10.433,0.9986903667449951
6,fbd8dfb0-abba-4e9a-8342-eac2a8174395,PPA:491,2024-10-04 10:08:10.433,13.04999677730002
7,9a4fdfbf-ce2a-4e55-b2b7-5ab7e7322b17,PPA:490,2024-10-04 10:08:10.433,0.9997492432594299
8,0435b797-03ec-4d21-8273-3157e22b5a55,PPA:493,2024-10-04 10:08:10.433,13.039997380355121
9,9f4c9ab9-74b8-4d9a-a0fb-2b3328f38e20,PPA:492,2024-10-04 10:08:10.433,0.9948765635490417
